In [0]:
### Code Not In Use Because Found Better Options Than Developing these one


### Here Lucid already developed better options in modelzoo module, however you can learn from these ...

### Creating TensorFlow session and loading the model from previous saved model.
##
## Examples:
##  - graphdef_path = '/content/model/Model.pb'
##  - graphdef_path = '/content/tensorflow_inception_graph.pb'
##
### Something is not quite right with show_graph and returning graph_def from a graph context or session context
### You can just use Lucid interface to do it, but here is a small hack handle.

def load_model(graphdef_path):

  with TF.Graph().as_default() as graph:  
  
    with TF.gfile.GFile(graphdef_path, 'rb') as f, TF.InteractiveSession(graph=graph).as_default() as sess:
      graph_def = TF.GraphDef()
      graph_def.ParseFromString(f.read())
    t_input = TF.placeholder(np.float32, name='input_1') # define the input tensor
    imagenet_mean = 117.0
    t_preprocessed = TF.expand_dims(t_input-imagenet_mean, 0)
    TF.import_graph_def(graph_def, {'input_1':t_preprocessed} )
    show_graph(graph_def)
    
    #return graph_def    
    
# Use Tensorboard API and ipython web capabilities to display graph in the notebook.
      
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(graph_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    
# Helper functions to show the graphs in the notebook

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = TF.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = TF.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = TF.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def